# 1. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 2. Importing data

In [2]:
#create path
path = r'C:\Users\Mathilde\Documents\DATA ANALYSIS CAREERFOUNDRY\May 2024 - Instacart Basket Analysis'
#orders data
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Original data', 'orders.csv'), index_col = False)
#products data
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original data', 'products.csv'), index_col = False)
#departments data
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Original data', 'departments.csv'), index_col = False)

# 3. Wrangling data

## 3.1. Orders data frame

In [3]:
#dropping eval_set column from orders data frame
df_ords=df_ords.drop(columns = ['eval_set'])

In [4]:
#changing data type for order and user ID in orders data frame
df_ords['order_id'] = df_ords['order_id'].astype('str')
df_ords['user_id'] = df_ords['user_id'].astype('str')

## 3.2. Departments data frame

In [5]:
#transposing df_dep
df_dep_t=df_dep.T

### Change header of departments df

In [6]:
#take 1st row of df_dep_t for the header
new_header = df_dep_t.iloc[0]

In [7]:
#copying df_dep_t from row 1 onward
df_dep_t_new = df_dep_t[1:]

In [8]:
#set the header row as the df header
df_dep_t_new.columns = new_header

### Checking result of the transformation

In [9]:
df_dep_t_new

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


#### Observation:

The data frame is no longer wide and the header has been fixed successfully.

# 4. Creating Data Dictionary

In [10]:
data_dict = df_dep_t_new.to_dict('index')

# 5. Task

In [11]:
# Step 2: it doesn't make sense to make stats on the order number as it is basically an ID (for each user, the order number is unique) so I am changing it to string.
df_ords['order_number'] = df_ords['order_number'].astype('str')

In [12]:
#checking if the type changed - doesn't appear in the stats so aok
df_ords.describe()

,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.214874e+06
mean,2.776219e+00,1.345202e+01,1.111484e+01
std,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+01,4.000000e+00
50%,3.000000e+00,1.300000e+01,7.000000e+00
75%,5.000000e+00,1.600000e+01,1.500000e+01
max,6.000000e+00,2.300000e+01,3.000000e+01


In [13]:
#Step 3: rename column day of the week in orders data frame (there is no other unintuitive name in the set so I used the same as in the exercise)
df_ords.rename(columns = {'order_dow' : 'orders_day_of_the_week'}, inplace = False)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


In [14]:
#Step 4: finding busiest hour
df_ords['order_hour_of_day'].value_counts(dropna = True)

order_hour_of_day
10    288418
11    284728
15    283639
14    283042
13    277999
12    272841
16    272553
9     257812
17    228795
18    182912
8     178201
19    140569
20    104292
7      91868
21     78109
22     61468
23     40043
6      30529
0      22758
1      12398
5       9569
2       7539
4       5527
3       5474
Name: count, dtype: int64

#### Observation:

The busiest time of the day to place orders is 10am, closely followed by 11am.

In [15]:
#Step 5: what "4" means in the data dictionary
print(data_dict.get('4'))

{'department': 'produce'}


#### Observation:

The department_id "4" corresponds to the category "produce".

In [16]:
#Step 6: Creating subset for breakfast items. First, identify breakfast dept id:
data_dict

{'1': {'department': 'frozen'},
 '2': {'department': 'other'},
 '3': {'department': 'bakery'},
 '4': {'department': 'produce'},
 '5': {'department': 'alcohol'},
 '6': {'department': 'international'},
 '7': {'department': 'beverages'},
 '8': {'department': 'pets'},
 '9': {'department': 'dry goods pasta'},
 '10': {'department': 'bulk'},
 '11': {'department': 'personal care'},
 '12': {'department': 'meat seafood'},
 '13': {'department': 'pantry'},
 '14': {'department': 'breakfast'},
 '15': {'department': 'canned goods'},
 '16': {'department': 'dairy eggs'},
 '17': {'department': 'household'},
 '18': {'department': 'babies'},
 '19': {'department': 'snacks'},
 '20': {'department': 'deli'},
 '21': {'department': 'missing'}}

-> department_id for "breakfast" is 14.

In [17]:
#creating the subset
df_prods_breakfast = df_prods.loc[df_prods['department_id'] == 14]

In [18]:
#checking result
df_prods_breakfast.head()

,product_id,product_name,aisle_id,department_id,prices
27,28,Wheat Chex Cereal,121,14,10.1
33,34,NaN,121,14,12.2
67,68,"Pancake Mix, Buttermilk",130,14,13.7
89,90,Smorz Cereal,121,14,3.9
210,211,Gluten Free Organic Cereal Coconut Maple Vanilla,130,14,3.6


Step 7: department_id for "alcohol", "deli", "bevrages" and "meat/seafood" are respectively 5, 20, 7 and 12.

In [19]:
#Step 7: creating "dinner party" subset.
df_prods_dinner_party = df_prods.loc[df_prods['department_id'].isin([5,7,12,20])]

#### Observation:

The sales team can use the data frame df_prods_dinner_party for their analysis.

In [20]:
#Step 8: find the number of rows
df_prods_dinner_party.shape

(7650, 5)

#### Observation:

The new dinner party data frame has 7650 rows.

In [21]:
#Step 9: finding info about customer with user_id 1
df_user1=df_ords.loc[df_ords['user_id'] == '1']
df_user1

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
5,3367565,1,6,2,7,19.0
6,550135,1,7,1,9,20.0
7,3108588,1,8,1,14,14.0
8,2295261,1,9,1,16,0.0
9,2550362,1,10,4,8,30.0


In [22]:
#Step 10: stats about the user's behaviour
df_user1.describe()

,order_dow,order_hour_of_day,days_since_prior_order
count,11.000000,11.000000,10.000000
mean,2.636364,10.090909,19.000000
std,1.286291,3.477198,9.030811
min,1.000000,7.000000,0.000000
25%,1.500000,7.500000,14.250000
50%,3.000000,8.000000,19.500000
75%,4.000000,13.000000,26.250000
max,4.000000,16.000000,30.000000


#### Observation:

This user only places orders from Sunday to Wednesday, never from Thrusday to Saturday. More than half of the time, he places orders at 8am or earlier. He places orders on average every 19 days. He never spend more than 30 days without placing orders , has already placed 2 orders the same day.

# 6. Export Data

In [23]:
#export orders data frame
df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_wrangled.csv'))

In [24]:
#export departments data frame
df_dep_t_new.to_csv(os.path.join(path, '02 Data','Prepared Data', 'departments_wrangled.csv'))